In [4]:
#|default_exp utils.clean_mem

# Memory Management Utilities

This notebook helps you monitor and clean up memory usage from Jupyter notebooks and models.


## 1. Check Current Notebook Memory Usage


In [3]:
#| export

import os
import psutil
import sys

def get_current_notebook_memory():
    """Get memory usage of the current notebook/kernel"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    mem_mb = mem_info.rss / 1024 / 1024  # Convert to MB
    mem_gb = mem_mb / 1024
    
    print(f"Current Notebook (PID: {os.getpid()}):")
    print(f"  Memory Usage: {mem_mb:.2f} MB ({mem_gb:.2f} GB)")
    print(f"  CPU Usage: {process.cpu_percent(interval=1):.1f}%")
    
    # Show largest variables in memory
    print("\nLargest variables in memory:")
    vars_size = [(name, sys.getsizeof(obj) / (1024**2)) 
                 for name, obj in globals().items() 
                 if not name.startswith('_')]
    vars_size.sort(key=lambda x: x[1], reverse=True)
    
    for name, size_mb in vars_size[:10]:
        if size_mb > 0.1:  # Only show if > 0.1 MB
            print(f"  {name}: {size_mb:.2f} MB")
    
    return mem_gb

In [4]:
get_current_notebook_memory()

Current Notebook (PID: 5285):
  Memory Usage: 80.48 MB (0.08 GB)
  CPU Usage: 0.1%

Largest variables in memory:


0.0785980224609375

## 2. Check All Jupyter Kernels Memory Usage


In [5]:
#| export

import subprocess
from datetime import datetime

def get_all_kernels_memory():
    """Get memory usage of all running Jupyter kernels"""
    kernels = []
    total_mem_mb = 0
    
    try:
        # Get all ipykernel processes
        result = subprocess.run(
            ['ps', 'aux'], 
            capture_output=True, 
            text=True
        )
        
        lines = result.stdout.split('\n')
        current_pid = os.getpid()
        
        print("Running Jupyter Kernels:")
        print("-" * 90)
        print(f"{'PID':<8} {'Memory (MB)':<15} {'Memory (GB)':<15} {'CPU%':<8} {'Age':<15} {'Current'}")
        print("-" * 90)
        
        for line in lines:
            if 'ipykernel_launcher' in line and 'grep' not in line:
                parts = line.split()
                pid = int(parts[1])
                mem_kb = float(parts[5])
                mem_mb = mem_kb / 1024
                mem_gb = mem_mb / 1024
                cpu_percent = parts[2]
                
                total_mem_mb += mem_mb
                
                try:
                    proc = psutil.Process(pid)
                    create_time = datetime.fromtimestamp(proc.create_time())
                    age = datetime.now() - create_time
                    
                    if age.days > 0:
                        age_str = f"{age.days}d {age.seconds//3600}h"
                    elif age.seconds > 3600:
                        age_str = f"{age.seconds//3600}h {(age.seconds%3600)//60}m"
                    else:
                        age_str = f"{age.seconds//60}m {age.seconds%60}s"
                    
                    is_current = "←  YOU" if pid == current_pid else ""
                    
                    print(f"{pid:<8} {mem_mb:<15.2f} {mem_gb:<15.3f} {cpu_percent:<8} {age_str:<15} {is_current}")
                    
                    kernels.append({
                        'pid': pid,
                        'memory_mb': mem_mb,
                        'memory_gb': mem_gb,
                        'age': age_str,
                        'is_current': pid == current_pid
                    })
                except:
                    pass
        
        total_gb = total_mem_mb / 1024
        print("-" * 90)
        print(f"Total: {len(kernels)} kernels using {total_mem_mb:.2f} MB ({total_gb:.2f} GB)")
        print(f"Note: This is just process memory. Models/tensors can use much more!")
        
        return kernels
        
    except Exception as e:
        print(f"Error: {e}")
        return []


In [6]:
kernels = get_all_kernels_memory()

Running Jupyter Kernels:
------------------------------------------------------------------------------------------
PID      Memory (MB)     Memory (GB)     CPU%     Age             Current
------------------------------------------------------------------------------------------
61565    15.50           0.015           0.0      21h 45m         
40951    62.64           0.061           0.0      1d 4h           
90514    64.23           0.063           0.0      1d 8h           
37766    23.66           0.023           0.0      2d 3h           
8899     65.12           0.064           0.0      2d 8h           
5803     15.45           0.015           0.0      2d 8h           
96616    15.31           0.015           0.0      2d 8h           
96614    15.34           0.015           0.0      2d 8h           
87348    15.39           0.015           0.0      2d 8h           
64897    15.44           0.015           0.0      2d 8h           
5285     80.61           0.079           0.0     

## 3. Check System Memory Status


In [13]:
#| export

def get_system_memory():
    """Get overall system memory status"""
    mem = psutil.virtual_memory()
    swap = psutil.swap_memory()
    
    print("System Memory Status:")
    print("=" * 60)
    print(f"Total RAM:      {mem.total / (1024**3):>10.2f} GB")
    print(f"Available:      {mem.available / (1024**3):>10.2f} GB")
    print(f"Used:           {mem.used / (1024**3):>10.2f} GB")
    print(f"Free:           {mem.free / (1024**3):>10.2f} GB")
    print(f"Usage:          {mem.percent:>10.1f}%")
    print()
    print(f"Swap Total:     {swap.total / (1024**3):>10.2f} GB")
    print(f"Swap Used:      {swap.used / (1024**3):>10.2f} GB")
    print(f"Swap Free:      {swap.free / (1024**3):>10.2f} GB")
    print(f"Swap Usage:     {swap.percent:>10.1f}%")
    print("=" * 60)
    
    # Memory pressure indicator
    if mem.percent > 90:
        print("⚠️  CRITICAL: Memory usage is very high!")
    elif mem.percent > 75:
        print("⚠️  WARNING: Memory usage is high")
    elif mem.percent > 50:
        print("✓  Memory usage is moderate")
    else:
        print("✓  Memory usage is healthy")
    
    return mem


In [14]:
mem = get_system_memory()

System Memory Status:
Total RAM:          128.00 GB
Available:           27.66 GB
Used:                35.55 GB
Free:                 0.13 GB
Usage:                78.4%

Swap Total:          42.00 GB
Swap Used:           41.28 GB
Swap Free:            0.72 GB
Swap Usage:           98.3%
⚠️  WARNING: Memory usage is high


## 4. Clean Current Notebook Memory


In [15]:
#| export

import gc

def clean_current_notebook(keep_functions=True):
    """Clean up memory in the current notebook
    
    Args:
        keep_functions: If True, keeps function definitions
    """
    print("Before cleanup:")
    mem_before = get_current_notebook_memory()
    
    # Get list of variables to delete
    to_delete = []
    for name in list(globals().keys()):
        if name.startswith('_'):
            continue
        if keep_functions and callable(globals()[name]) and not isinstance(globals()[name], type):
            continue
        if name in ['get_current_notebook_memory', 'get_all_kernels_memory', 
                    'get_system_memory', 'clean_current_notebook', 
                    'kill_specific_kernel', 'kill_all_kernels',
                    'os', 'psutil', 'sys', 'subprocess', 'datetime', 'gc']:
            continue
        to_delete.append(name)
    
    # Delete variables
    for name in to_delete:
        if name in globals():
            del globals()[name]
    
    # Try to clear PyTorch cache if available
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("  ✓ Cleared CUDA cache")
    except:
        pass
    
    # Garbage collection
    collected = gc.collect()
    
    print(f"\n✓ Deleted {len(to_delete)} variables")
    print(f"✓ Garbage collector freed {collected} objects")
    
    print("\nAfter cleanup:")
    mem_after = get_current_notebook_memory()
    
    print(f"\n{'='*60}")
    print(f"Memory freed: {(mem_before - mem_after)*1024:.2f} MB")
    print(f"{'='*60}")


In [16]:
# Uncomment to run:
# clean_current_notebook(keep_functions=True)

## 5. Kill Specific Kernel(s) by PID


In [17]:
#| export

def kill_specific_kernel(pid):
    """Kill a specific Jupyter kernel by PID
    
    Args:
        pid: Process ID to kill (int)
    """
    current_pid = os.getpid()
    
    if pid == current_pid:
        print(f"⚠️  Warning: PID {pid} is YOUR current notebook!")
        response = input("Are you sure you want to kill your own kernel? (yes/no): ")
        if response.lower() != 'yes':
            print("Cancelled.")
            return
    
    try:
        proc = psutil.Process(pid)
        mem_mb = proc.memory_info().rss / (1024**2)
        
        print(f"Killing kernel PID {pid} (using {mem_mb:.2f} MB)...")
        proc.terminate()
        proc.wait(timeout=3)
        print(f"✓ Successfully killed kernel {pid}")
        
    except psutil.NoSuchProcess:
        print(f"⚠️  Process {pid} not found")
    except Exception as e:
        print(f"Error: {e}")


In [18]:
# Example usage:
# kill_specific_kernel(12345)  # Replace with actual PID from the list above

## 6. Kill All Kernels (Except Current)


In [19]:
#| export

def kill_all_kernels(exclude_current=True):
    """Kill all Jupyter kernels
    
    Args:
        exclude_current: If True, don't kill the current notebook
    """
    current_pid = os.getpid()
    killed = []
    
    try:
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
        lines = result.stdout.split('\n')
        
        for line in lines:
            if 'ipykernel_launcher' in line and 'grep' not in line:
                parts = line.split()
                pid = int(parts[1])
                
                if exclude_current and pid == current_pid:
                    print(f"Skipping current notebook (PID: {pid})")
                    continue
                
                try:
                    proc = psutil.Process(pid)
                    mem_mb = proc.memory_info().rss / (1024**2)
                    proc.terminate()
                    killed.append((pid, mem_mb))
                    print(f"✓ Killed kernel {pid} (was using {mem_mb:.2f} MB)")
                except:
                    pass
        
        if killed:
            total_freed = sum(mem for _, mem in killed)
            print(f"\n{'='*60}")
            print(f"✓ Killed {len(killed)} kernel(s)")
            print(f"✓ Freed approximately {total_freed:.2f} MB of process memory")
            print(f"{'='*60}")
        else:
            print("No other kernels found to kill.")
            
    except Exception as e:
        print(f"Error: {e}")


In [20]:
# Uncomment to kill all other kernels:
# kill_all_kernels(exclude_current=True)

# Uncomment to kill ALL kernels including this one (will crash this notebook):
# kill_all_kernels(exclude_current=False)

## 7. Quick Commands (Run these as needed)


In [7]:
# Quick memory check - run this anytime
print("="*60)
get_system_memory()
print()
kernels = get_all_kernels_memory()


System Memory Status:
Total RAM:          128.00 GB
Available:           24.33 GB
Used:                32.15 GB
Free:                 0.06 GB
Usage:                81.0%

Swap Total:          42.00 GB
Swap Used:           40.94 GB
Swap Free:            1.06 GB
Swap Usage:           97.5%
⚠️  WARNING: Memory usage is high

Running Jupyter Kernels:
------------------------------------------------------------------------------------------
PID      Memory (MB)     Memory (GB)     CPU%     Age             Current
------------------------------------------------------------------------------------------
98644    518.12          0.506           0.0      11m 13s         
85984    72.61           0.071           0.0      12m 47s         
31242    37.70           0.037           0.0      6m 18s          ←  YOU
------------------------------------------------------------------------------------------
Total: 3 kernels using 628.44 MB (0.61 GB)
Note: This is just process memory. Models/tensors can 

In [21]:
#| export

# Check specific variable sizes (useful for large models)
def check_variable_sizes(min_mb=10):
    """Show all variables larger than min_mb in current notebook"""
    print(f"Variables larger than {min_mb} MB:")
    print("-" * 50)
    
    import sys
    vars_size = []
    for name, obj in globals().items():
        if not name.startswith('_'):
            size_mb = sys.getsizeof(obj) / (1024**2)
            if size_mb > min_mb:
                vars_size.append((name, size_mb, type(obj).__name__))
    
    vars_size.sort(key=lambda x: x[1], reverse=True)
    
    for name, size_mb, obj_type in vars_size:
        print(f"{name:<20} {size_mb:>10.2f} MB  ({obj_type})")
    
    if not vars_size:
        print(f"No variables found > {min_mb} MB")


In [22]:
# Check for large variables
check_variable_sizes(min_mb=10)

Variables larger than 10 MB:
--------------------------------------------------
No variables found > 10 MB


## 8. Emergency: Kill Everything from Terminal

If Jupyter becomes unresponsive, run this in your terminal:

```bash
pkill -f "ipykernel_launcher"
```

Or add aliases to your `~/.zshrc`:

```bash
alias kernels="ps aux | grep ipykernel_launcher | grep -v grep | wc -l"
alias killkernels="pkill -f ipykernel_launcher"
alias memstat="vm_stat | perl -ne '/page size of (\d+)/ and \$size=\$1; /Pages\s+([^:]+)[^\d]+(\d+)/ and printf(\"%-16s % 16.2f Mi\n\", \"\$1:\", \$2 * \$size / 1048576);'"
```


In [5]:
import nbdev; nbdev.nbdev_export()